In [1]:
# !sudo apt update
# !sudo apt -y install build-essential vim
# !pip3 install networkx node2vec python-Levenshtein nodevectors emblaze pymde

import pandas as pd
import json
import networkx as nx
from node2vec import Node2Vec
from gensim.models import KeyedVectors
import numpy as np
from collections import defaultdict

In [22]:
all_animes_df = pd.read_csv('/data/jx4237data/projects/web_page/sprout-main/work/data/processed-metadata.csv')
all_animes_df['average_rating'] = all_animes_df['score']
all_animes_df.columns

new_order = [
    'id', 'title', 'title_english', 'title_japanese', 'title_synonyms',
    'aired_from_year', 'rating_count', 'average_rating', 'image_url', 'type', 'source', 'episodes', 'status', 'airing',
    'aired_string', 'aired', 'duration', 'rating', 'score', 
    'rank', 'popularity', 'members', 'favorites', 'background', 'premiered',
    'broadcast', 'related_anime', 'producer', 'licensor', 'studio', 'genre',
    'opening_theme', 'ending_theme', 'aired_from_year_string'
]
# Reindex the DataFrame with the new order of columns
all_animes_df = all_animes_df.reindex(columns=new_order)
all_animes_df.columns

Index(['id', 'title', 'title_english', 'title_japanese', 'title_synonyms',
       'aired_from_year', 'rating_count', 'average_rating', 'image_url',
       'type', 'source', 'episodes', 'status', 'airing', 'aired_string',
       'aired', 'duration', 'rating', 'score', 'rank', 'popularity', 'members',
       'favorites', 'background', 'premiered', 'broadcast', 'related_anime',
       'producer', 'licensor', 'studio', 'genre', 'opening_theme',
       'ending_theme', 'aired_from_year_string'],
      dtype='object')

In [23]:
all_animes_df.to_csv('/data/jx4237data/projects/web_page/sprout-main/work/data/processed-metadata.csv', index=False)

In [8]:
all_animes = []
anime_title_by_id = {}
anime_id_by_title = {}
anime_ix_by_id = {}
anime_by_id = {}

i = 0
for anime in all_animes_df.itertuples(index=False):
    all_animes.append(anime)
    anime_title_by_id[anime.id] = anime.title
    anime_id_by_title[anime.title] = anime.id
    anime_ix_by_id[anime.id] = i
    anime_by_id[anime.id] = anime
    i += 1

0       2012
1       2007
2       2008
3       2002
4       2012
        ... 
9427    2018
9428    2015
9429    2017
9430    1987
9431    2010
Name: aired_from_year, Length: 9432, dtype: int64

In [3]:
all_ratings = pd.read_csv('/data/jx4237data/projects/web_page/sprout-main/work/data/collected_animelists.csv.gz', compression='gzip')

In [4]:
# only retain the "username", "anime_id", and "my_score" columns
ratings = all_ratings[['username', 'anime_id', 'my_score']]
# only retain rows where "my_score" is not null and greater than or equal to 6
ratings = ratings[ratings['my_score'].notnull()]
ratings = ratings[ratings['my_score'] > 0]
ratings = ratings[ratings['anime_id'].isin(anime_title_by_id.keys())]
# change to ix
ratings['anime_ix'] = ratings['anime_id'].apply(lambda x: anime_ix_by_id[x])

def scale_rating(rating: int) -> float:
    if rating == 10:
        return 10.1
    if rating == 9:
        return 9.1
    if rating == 8:
        return 6.5
    if rating == 7:
        return 3.3
    if rating == 6:
        return 0.5
    if rating == 5:
        return -1.5
    if rating == 4:
        return -4.5
    if rating == 3:
        return -7.5
    if rating == 2:
        return -10.5
    if rating == 1:
        return -13.5
    raise ValueError("Invalid rating: {}".format(rating))

# scale ratings from score to our custom scale
ratings['my_score'] = ratings['my_score'].apply(scale_rating)
# only keep high scores
ratings = ratings[ratings['my_score'] > 0]

print("Remaining ratings:", ratings.shape[0])
print(ratings.head())

# collect python garbage
import gc
gc.collect()

Remaining ratings: 39126493
   username  anime_id  my_score  anime_ix
0  karthiga        21       9.1        34
1  karthiga        59       3.3        15
2  karthiga        74       3.3        21
3  karthiga       120       3.3        32
4  karthiga       178       3.3         6


0

In [5]:
# ratings_subset = ratings.sample(n=5000000)
# ratings_subset = ratings[:5000000]
ratings_subset = ratings

# Retain only ratings of anime that have >=100 ratings
ratings_subset = ratings_subset[ratings['anime_id'].isin(anime_title_by_id.keys())]
print(f"Retained {ratings_subset.shape[0]} ratings out of {ratings.shape[0]} total")

Retained 39126493 ratings out of 39126493 total


In [6]:
all_ids = set(anime_ix_by_id.keys())
all_ixs = set(anime_ix_by_id.values())

print(f"ids: {len(all_ids)}")
print(f"ixs: {len(all_ixs)}")

ids: 9432
ixs: 9432


In [7]:
from collections import defaultdict

ratings_by_username = defaultdict(list)

i = 0
for row in ratings_subset.itertuples():
    if i % 1000000 == 0:
        print(f"Processed {i} rows")
    i += 1

    rating = row.my_score
    if rating < 0:
        continue
    anime_ix = anime_ix_by_id[row.anime_id]
    ratings_by_username[row.username].append((anime_ix, rating))

Processed 0 rows
Processed 1000000 rows
Processed 2000000 rows
Processed 3000000 rows
Processed 4000000 rows
Processed 5000000 rows
Processed 6000000 rows
Processed 7000000 rows
Processed 8000000 rows
Processed 9000000 rows
Processed 10000000 rows
Processed 11000000 rows
Processed 12000000 rows
Processed 13000000 rows
Processed 14000000 rows
Processed 15000000 rows
Processed 16000000 rows
Processed 17000000 rows
Processed 18000000 rows
Processed 19000000 rows
Processed 20000000 rows
Processed 21000000 rows
Processed 22000000 rows
Processed 23000000 rows
Processed 24000000 rows
Processed 25000000 rows
Processed 26000000 rows
Processed 27000000 rows
Processed 28000000 rows
Processed 29000000 rows
Processed 30000000 rows
Processed 31000000 rows
Processed 32000000 rows
Processed 33000000 rows
Processed 34000000 rows
Processed 35000000 rows
Processed 36000000 rows
Processed 37000000 rows
Processed 38000000 rows
Processed 39000000 rows


In [8]:
# for each user's ratings, deduplicate.
processed_rows = 0
for username in ratings_by_username.keys():
    before_len = len(ratings_by_username[username])
    ratings_by_username[username] = list(set(ratings_by_username[username]))
    after_len = len(ratings_by_username[username])
    if before_len != after_len:
        print(f"Removed {before_len - after_len} duplicate ratings for user {username}")
    processed_rows += 1
    if processed_rows % 500000 == 0:
        print(f"Processed {processed_rows} rows")

In [9]:
ratings = ratings = [(np.asarray([anime_ix for anime_ix, rating in ratings]), np.asarray([rating for anime_ix, rating in ratings])) for ratings in ratings_by_username.values()]

In [10]:
import numpy as np
from numba import njit, prange
from numba.np.ufunc import parallel

@njit(parallel=True, nopython=True)
def compute_cooccurrence_matrix(anime_count: int, ratings: list[(np.array, np.array)]) -> np.ndarray:
    cooccurrence_matrix = np.zeros((anime_count, anime_count))

    for anime_indices, ratings in ratings:
        rating_count_for_user = len(anime_indices)

        for i in prange(rating_count_for_user):
            for j in range(rating_count_for_user):
                if i == j:
                    continue

                cooccurrence_matrix[anime_indices[i], anime_indices[j]] += ratings[i] * ratings[j]

    return cooccurrence_matrix

print(f"Users to process: {len(ratings)}")
cooccurrence_matrix = compute_cooccurrence_matrix(len(anime_title_by_id), ratings)

compute_cooccurrence_matrix.parallel_diagnostics(level=4)

/data/jx4237data/python_envs/sprout-env/lib/python3.10/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


Users to process: 270944
 
 Parallel Accelerator Optimizing:  Function compute_cooccurrence_matrix, 
/tmp/ipykernel_2488519/1771784096.py (5)  


Parallel loop listing for  Function compute_cooccurrence_matrix, /tmp/ipykernel_2488519/1771784096.py (5) 
---------------------------------------------------------------------------------------------------------|loop #ID
@njit(parallel=True, nopython=True)                                                                      | 
def compute_cooccurrence_matrix(anime_count: int, ratings: list[(np.array, np.array)]) -> np.ndarray:    | 
    cooccurrence_matrix = np.zeros((anime_count, anime_count))-------------------------------------------| #0
                                                                                                         | 
    for anime_indices, ratings in ratings:                                                               | 
        rating_count_for_user = len(anime_indices)                                        

In [11]:
cooccurrence_matrix[0][1]

323179.98000001477

In [12]:
# Save the co-occurrence matrix to compressed binary numpy file
np.savez_compressed('/data/jx4237data/projects/web_page/sprout-main/work/data/cooccurrence_matrix.npz', cooccurrence_matrix)

In [13]:
cooccurrence_matrix = np.load('/data/jx4237data/projects/web_page/sprout-main/work/data/cooccurrence_matrix.npz')['arr_0']

In [14]:
import json
import math

# def score_num_recommendations(num_recommendations):
#     return math.sqrt(num_recommendations) / 2.

def parse_extra(item):
    related = json.loads(item.related_anime) 
    related = [(r['node']['id'], 0.2 if r['relation_type'] == 'character' else 40) for r in related]
    # recommended = [(r['node']['id'], score_num_recommendations(r['num_recommendations'])) for r in json.loads(item.recommendations)]
    # return [(anime_ix_by_id.get(id), score) for id, score in related + recommended if anime_ix_by_id.get(id) is not None]
    return [(anime_ix_by_id.get(id), score) for id, score in related if anime_ix_by_id.get(id) is not None]


related_by_anime_ix = defaultdict(lambda: defaultdict(float))

for item in all_animes:
    anime_ix = anime_ix_by_id.get(item.id)
    if anime_ix is None:
        continue
    related_for_item = parse_extra(item)
    entry = related_by_anime_ix[anime_ix]
    for ix, weight in related_for_item:
        entry[ix] += weight
        
kill_me_baby_id = 11079
kill_me_baby_ix = anime_ix_by_id[kill_me_baby_id]
[(score, all_animes[ix].title) for ix, score in related_by_anime_ix[kill_me_baby_ix].items()]

[(40.0, 'Kill Me Baby: Butsuzou Kegatte Nise Halloween')]

In [15]:
cooccurrence_matrix_wextra = cooccurrence_matrix.copy()

for anime_0_ix in range(len(cooccurrence_matrix)):
    related_for_anime0 = related_by_anime_ix[anime_0_ix]
    max_weight = cooccurrence_matrix[anime_0_ix].max()

    for ix, weight in related_for_anime0.items():
        if ix == anime_0_ix:
            continue
        cooccurrence_matrix_wextra[anime_0_ix, ix] += weight * max_weight

np.save('/data/jx4237data/projects/web_page/sprout-main/work/data/cooccurrence_matrix_wextra.npy', cooccurrence_matrix_wextra)

In [16]:
cooccurrence_matrix_wextra[0][1]

323179.98000001477

In [17]:
graph = nx.Graph()

def get_topn(weights, topn):
    return sorted(enumerate(weights), key=lambda x: x[1], reverse=True)[:topn]

topn = 40

def get_extra_topn(rating_count):
    return int(max(math.pow(rating_count, 0.46) * 0.4 - 10, 0))

for anime_0_ix in range(cooccurrence_matrix.shape[0]):
    row = cooccurrence_matrix[anime_0_ix]
    top_edges = get_topn(row, topn)
    top_related_anime_ixs_set = set([ix for ix, weight in top_edges])

    top_magnitude = 0 if len(top_edges) == 0 else top_edges[0][1]
    if top_magnitude == 0:
        continue
    related_for_anime0 = related_by_anime_ix[anime_0_ix]

    for related_anime_ix, related_weight in related_for_anime0.items():
        if related_anime_ix in top_related_anime_ixs_set:
            continue
        top_edges.append((related_anime_ix, 0.))

    for anime_1_ix, base_weight in top_edges:
        extra_weight = related_for_anime0[anime_1_ix]
        # This may need to be tuned
        weight = base_weight + extra_weight * top_magnitude * 1.
        graph.add_edge(anime_0_ix, anime_1_ix, weight=weight)

print("Edges:", graph.number_of_edges())

Edges: 370659


In [18]:
import csrgraph as cg
import nodevectors

cgraph = cg.csrgraph(graph)

In [19]:
dimensions = 10
order = 2
algo = "ggvec"

embedding_model = None
if algo == "ggvec":
    embedding_model = nodevectors.GGVec(n_components=dimensions, learning_rate=0.01, negative_ratio=0.6, verbose=True, order=order)
elif algo == "node2vec":
    embedding_model = nodevectors.Node2Vec(walklen=8, epochs=50, return_weight=2., neighbor_weight=1., n_components=dimensions, threads=14)
elif algo == "ProNE":
    embedding_model = nodevectors.ProNE(n_components=dimensions)

embeddings = embedding_model.fit_transform(cgraph)

Loss: 0.0048	:  12%|█▏        | 43/350 [00:40<04:47,  1.07it/s]

Converged! Loss: 0.0047


In [20]:
keys = [int(n) for n in cgraph.nodes()]

print(f"key length: {len(keys)}; weight length: {len(embeddings)}; anime_count: {len(all_animes)}")

fname = f"/data/jx4237data/projects/web_page/sprout-main/work/data/embedding_{algo}_full_posonly_wextra_top{topn}_{dimensions}d_order{order}.w2v"
with open(fname, 'wt') as f:
    tab = ' '
    nl = '\n'
    f.write(f"{len(keys)}{tab}{dimensions}{nl}")
    for key, embedding in zip(keys, embeddings):
        f.write(f"{key}{tab}{tab.join(map(str, embedding))}{nl}")
fname

key length: 9359; weight length: 9359; anime_count: 9432


'/data/jx4237data/projects/web_page/sprout-main/work/data/embedding_ggvec_full_posonly_wextra_top40_10d_order2.w2v'